# EAA Winter School in Computational Acoustics 2018


## Hierarchical elements

The objective of this practice is to present hierarchical elements 


### Preamble

We begin by loading a number of standard modules such as `numpy` and `matplotlib`. We also adjust some default parameters of figures.

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eig
from scipy.sparse.linalg import eigs
from scipy.special import legendre